### kk's notebook to download and merge mapPLUTO, Census Tract shapefile, and Census Tract population, and complaints

In [42]:
import pandas as pd
import zipfile
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon

### download pluto and concatenate into one dataframe

In [2]:
boroughs = ['Staten_Island', 'Queens', 'Manhattan', 'Bronx', 'Brooklyn']
abreevs = ['SI', 'QN', 'MN', 'BX', 'BK']

#boroughs = ['Staten_Island', 'Queens']
#abreevs = ['SI', 'QN']

pluto = []
i = 0
for elem in boroughs:
    temp = gpd.read_file('../data/'+elem+'/'+abreevs[i]+'MapPLUTO.shp')
    temp['BBL'] = temp['BBL'].astype(str)
    temp = temp.to_crs({'init':'epsg:4326'})
    pluto.append(temp)
    i+=1

In [3]:
p = pd.concat(pluto)

In [4]:
type(p)

geopandas.geodataframe.GeoDataFrame

#### download census tract shapefile

In [5]:
ct = gpd.read_file('../Data/cb_2015_36_tract_500k/cb_2015_36_tract_500k.shp')

In [6]:
ct.head(2)

,AFFGEOID,ALAND,AWATER,COUNTYFP,GEOID,LSAD,NAME,STATEFP,TRACTCE,geometry
0,1400000US36001000502,777446,0,001,36001000502,CT,5.02,36,000502,"POLYGON ((-73.79496 42.66871, -73.790798999999..."
1,1400000US36001013507,5575232,2491,001,36001013507,CT,135.07,36,013507,"POLYGON ((-73.802734 42.763943, -73.796492 42...."


In [7]:
ct = ct[['GEOID','geometry']]

In [8]:
print ct.crs
print p.crs

{'init': u'epsg:4269'}
{'init': 'epsg:4326'}


In [9]:
ct = ct.to_crs({'init':'epsg:4326'})

In [10]:
print ct.crs

{'init': 'epsg:4326'}


In [11]:
p = p[['BBL', 'AssessTot', 'BldgArea', 'BldgClass', 'BldgDepth', 'BldgFront', 'ComArea', \
              'FactryArea', 'GarageArea', 'LandUse', 'NumBldgs','NumFloors', 'OfficeArea', 'OtherArea', 'StrgeArea', \
              'StrgeArea', 'YearBuilt', 'geometry']]
p.head(2)

,BBL,AssessTot,BldgArea,BldgClass,BldgDepth,BldgFront,ComArea,FactryArea,GarageArea,LandUse,NumBldgs,NumFloors,OfficeArea,OtherArea,StrgeArea,StrgeArea,YearBuilt,geometry
0,5007470028.0,23400.0,1488,B2,62.0,24.0,0,0,0,01,1,1.0,0,0,0,0,1960,POLYGON ((-74.14250944979312 40.60980609846242...
1,5006230252.0,21518.0,0,V0,0.0,0.0,0,0,0,11,0,0.0,0,0,0,0,0,POLYGON ((-74.08780465394567 40.61401983466899...


In [13]:
len(p)

857513

#### reduce pluto dataframe for spatial join with census tract

In [14]:
pl = p[['BBL', 'geometry']]

In [15]:
len(pl.BBL.unique())

857513

In [16]:
pl = pl.reset_index()
pl = pl.drop('index',axis=1)
pl.tail()

,BBL,geometry
857508,3089460004.0,POLYGON ((-73.93001432912571 40.59472575579335...
857509,3089550176.0,POLYGON ((-73.93028289868332 40.59611436597339...
857510,3089550242.0,POLYGON ((-73.93059055424578 40.59559859964976...
857511,3089550278.0,POLYGON ((-73.93122617765142 40.59581280826773...
857512,3089550374.0,POLYGON ((-73.93188566121631 40.59525767836104...


In [17]:
ct.head(2)

,GEOID,geometry
0,36001000502,"POLYGON ((-73.79496 42.66871, -73.790798999999..."
1,36001013507,"POLYGON ((-73.802734 42.763943, -73.796492 42...."


#### join pluto and census tract using r-tree

In [18]:
#http://geoffboeing.com/2016/10/r-tree-spatial-index-python/
precise_matches = {}
spatial_index = pl.sindex
for i, GEOID in enumerate(ct.GEOID):
    possible_matches_index = list(spatial_index.intersection(ct['geometry'][i].bounds))
    possible_matches = pl.iloc[possible_matches_index]
    precise_matches[GEOID] = possible_matches[possible_matches.intersects(ct['geometry'][i])]

In [19]:
bbl_ct = pd.concat(precise_matches).reset_index()

In [22]:
bbl_ct.tail(2)

,level_0,level_1,BBL,geometry
864871,36119005500,573548,2056530067.0,POLYGON ((-73.81929087211066 40.89010449933943...
864872,36119005500,574300,2056530074.0,POLYGON ((-73.81879174801337 40.88997792167392...


In [24]:
print len(bbl_ct.level_0.unique())
print len(ct.GEOID.unique())

2187
4906


In [25]:
bbl_ct = bbl_ct.rename(columns={'level_0':'GEOID'})

In [26]:
bbl_ct.head(2)

,GEOID,level_1,BBL,geometry
0,36005000100,503649,2026050037.0,"POLYGON ((-73.89676470030579 40.7961948901257,..."
1,36005000100,498939,2026050040.0,POLYGON ((-73.88883042913881 40.79798044396698...


In [27]:
bbl_ct = bbl_ct.drop(['geometry', 'level_1'],axis=1)

In [205]:
print bbl_ct.BBL.dtypes
print p.BBL.dtypes

float64
float64


In [28]:
pluto_ct = pd.merge(p, bbl_ct, on='BBL')

In [29]:
print len(pluto_ct.BBL.unique())
print len(bbl_ct.BBL.unique())
print len(p.BBL.unique())

857348
857348
857513


In [30]:
pluto_ct = pluto_ct.drop('LandUse',axis=1)
pluto_ct = pluto_ct[pluto_ct['YearBuilt']>0]
pluto_ct = pluto_ct.drop('geometry',axis=1)

In [31]:
pluto_ct.to_csv('../data/ct_bbl.csv')

In [33]:
pluto_ct = pluto_ct.reset_index()
pluto_ct= pluto_ct.drop('index',axis=1)
pluto_ct.head(2)

,BBL,AssessTot,BldgArea,BldgClass,BldgDepth,BldgFront,ComArea,FactryArea,GarageArea,NumBldgs,NumFloors,OfficeArea,OtherArea,StrgeArea,StrgeArea,YearBuilt,GEOID
0,5007470028.0,23400.0,1488,B2,62.0,24.0,0,0,0,1,1.0,0,0,0,0,1960,36085018901
1,5007130017.0,22602.0,1316,A1,27.0,17.0,0,0,0,2,2.5,0,0,0,0,1920,36085018701


In [34]:
pluto_ct['BldgAge'] = 2017-pluto_ct['YearBuilt']
pluto_ct = pluto_ct.drop('YearBuilt',axis=1)

In [35]:
avg = pd.DataFrame(pluto_ct[['BldgAge','NumFloors','AssessTot','BldgArea','BldgDepth','BldgFront','ComArea',\
                           'FactryArea','GarageArea','NumBldgs',\
       'OfficeArea','OtherArea','StrgeArea']].groupby(pluto_ct['GEOID']).mean())

In [36]:
avg = avg.reset_index()

In [37]:
avg = avg.drop('index',axis=1)
print len(avg.GEOID.unique())
avg.head(2)

2180


,GEOID,BldgAge,NumFloors,AssessTot,BldgArea,BldgDepth,BldgFront,ComArea,FactryArea,GarageArea,NumBldgs,OfficeArea,OtherArea,StrgeArea,StrgeArea
0,36005000100,84.000000,6.00000,4.743675e+08,5.502107e+06,0.000000,0.000000,5.502107e+06,0.000000,0.000000,167.000000,0.000000,5.502107e+06,0.000000,0.000000
1,36005000200,65.569918,2.00765,2.326738e+04,2.008622e+03,43.002256,21.678966,1.139918e+02,26.983549,14.090482,1.177438,1.645123,4.275206e+01,4.301998,4.301998


In [38]:
avg.to_csv('../data/ct_bbl_avg.csv')

#### import complaint data

In [39]:
c = pd.read_csv('/Users/kristikorsberg/Downloads/311_Service_Requests_from_2010_to_Present.csv')

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (8,17,40,43,44,45,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
c = c[['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Complaint Type', 'Descriptor', 'Incident Zip', \
               'X Coordinate (State Plane)', 'Y Coordinate (State Plane)']]
c = c.dropna()
c.head(2)

,Unique Key,Created Date,Closed Date,Agency,Complaint Type,Descriptor,Incident Zip,X Coordinate (State Plane),Y Coordinate (State Plane)
0,35607069,03/02/2017 03:33:17 PM,03/12/2017 11:21:04 AM,HPD,FLOORING/STAIRS,FLOOR,11220,978923.0,172478.0
1,35597174,03/01/2017 05:10:34 PM,03/12/2017 09:40:02 AM,HPD,DOOR/WINDOW,DOOR,10467,1018715.0,258823.0


In [43]:
geometry = [Point(xy) for xy in zip(c['X Coordinate (State Plane)'], c['Y Coordinate (State Plane)'])]
c = c.drop(['X Coordinate (State Plane)', 'Y Coordinate (State Plane)'], axis=1)
crs = {'init': 'epsg:2263'}
c = gpd.GeoDataFrame(c, crs=crs, geometry=geometry)

In [47]:
print c.crs
print ct.crs

{'init': 'epsg:2263'}
{'init': 'epsg:4326'}


In [48]:
ct = ct.to_crs({'init':'epsg:2263'})

In [50]:
print c.crs
print ct.crs

{'init': 'epsg:2263'}
{'init': 'epsg:2263'}


In [51]:
#http://geoffboeing.com/2016/10/r-tree-spatial-index-python/
precise_matches = {}
spatial_index = c.sindex
for i, GEOID in enumerate(ct.GEOID):
    possible_matches_index = list(spatial_index.intersection(ct['geometry'][i].bounds))
    possible_matches = c.iloc[possible_matches_index]
    precise_matches[GEOID] = possible_matches[possible_matches.intersects(ct['geometry'][i])]

In [52]:
c_ct = pd.concat(precise_matches).reset_index()
c_ct.head(2)

,level_0,level_1,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry
0,36005000100,155852,DOHMH,04/07/2017 12:00:01 AM,Food Poisoning,03/29/2017 12:00:00 AM,1 or 2,11370,35817710.0,POINT (1016258 226737)
1,36005000100,183092,DOT,04/14/2017 12:42:42 PM,Sidewalk Condition,04/01/2017 12:21:09 PM,Sidewalk Collapsed,11370,35844184.0,POINT (1016332 228466)


In [54]:
c_ct = c_ct.rename(columns={'level_0':'GEOID'})
c_ct = c_ct.drop(['level_1', 'geometry'],axis=1)

In [55]:
c_ct = pd.DataFrame(c_ct['Unique Key'].groupby(c_ct['GEOID']).count())
c_ct = c_ct.reset_index()
c_ct.head(2)

,GEOID,Unique Key
0,36005000100,5
1,36005000200,132


In [56]:
c_ct = c_ct.rename(columns={'Unique Key':'Complaint Count'})

In [57]:
bbl_c_ct = pd.merge(c_ct, avg, on='GEOID', how='right')
print len(bbl_c_ct.GEOID.unique())
print bbl_c_ct.isnull().sum()

2180
GEOID               0
Complaint Count    20
BldgAge             0
NumFloors           0
AssessTot           0
BldgArea            0
BldgDepth           0
BldgFront           0
ComArea             0
FactryArea          0
GarageArea          0
NumBldgs            0
OfficeArea          0
OtherArea           0
StrgeArea           0
StrgeArea           0
dtype: int64


In [58]:
bbl_c_ct = bbl_c_ct.replace(np.nan,0)

#### import population data from census

In [59]:
pop = pd.read_csv('../data/ACS_15_5YR_B01003/ACS_15_5YR_B01003_with_ann.csv', header=1)

In [60]:
pop.head(2)

,Id,Id2,Geography,Estimate; Total,Margin of Error; Total
0,1400000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",7703,416
1,1400000US36005000200,36005000200,"Census Tract 2, Bronx County, New York",5403,632


In [61]:
print len(pop.Id2.unique())

2167


In [62]:
pop = pop.drop(['Id', 'Geography', 'Margin of Error; Total'],axis=1)
pop = pop.rename(columns={'Id2':'GEOID','Estimate; Total':'Population'})
pop.head(2)

,GEOID,Population
0,36005000100,7703
1,36005000200,5403


In [65]:
print pop.GEOID.dtypes
print bbl_c_ct.GEOID.dtypes

int64
object


In [66]:
bbl_c_ct.GEOID = bbl_c_ct.GEOID.astype(int)

In [67]:
bbl_c_ct_pop = pd.merge(bbl_c_ct, pop, on='GEOID', how='left')
print len(bbl_c_ct_pop.GEOID.unique())
print bbl_c_ct_pop.isnull().sum()

2180
GEOID               0
Complaint Count     0
BldgAge             0
NumFloors           0
AssessTot           0
BldgArea            0
BldgDepth           0
BldgFront           0
ComArea             0
FactryArea          0
GarageArea          0
NumBldgs            0
OfficeArea          0
OtherArea           0
StrgeArea           0
StrgeArea           0
Population         21
dtype: int64


In [68]:
bbl_c_ct_pop = bbl_c_ct_pop.replace(np.nan, 0)

In [69]:
bbl_c_ct_pop.head()

,GEOID,Complaint Count,BldgAge,NumFloors,AssessTot,BldgArea,BldgDepth,BldgFront,ComArea,FactryArea,GarageArea,NumBldgs,OfficeArea,OtherArea,StrgeArea,StrgeArea,Population
0,36005000100,5.0,84.000000,6.000000,4.743675e+08,5.502107e+06,0.000000,0.000000,5.502107e+06,0.000000,0.000000,167.000000,0.000000,5.502107e+06,0.000000,0.000000,7703.0
1,36005000200,132.0,65.569918,2.007650,2.326738e+04,2.008622e+03,43.002256,21.678966,1.139918e+02,26.983549,14.090482,1.177438,1.645123,4.275206e+01,4.301998,4.301998,5403.0
2,36005000400,128.0,56.640180,1.994648,6.383420e+04,3.534175e+03,43.732414,22.232864,2.712114e+02,0.000000,22.709145,1.746627,12.632684,1.879760e+02,12.143928,12.143928,5915.0
3,36005001600,108.0,57.886628,2.248314,3.560243e+05,1.091054e+04,47.917064,27.443924,4.594878e+03,0.000000,28.023256,1.215116,854.316860,3.650875e+03,0.203488,0.203488,5879.0
4,36005001900,196.0,85.969863,2.319178,7.526278e+05,2.133805e+04,94.043507,63.055233,1.836729e+04,9525.504110,1401.117808,1.295890,1287.221918,5.293890e+02,5195.495890,5195.495890,2591.0


In [70]:
bbl_c_ct_pop.to_csv('../data/bbl_c_ct_pop.csv')